In [50]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.orm import relationship, declarative_base

Base = declarative_base()

class Author(Base):
    __tablename__ = 'authors'
    id = Column(Integer, primary_key=True)
    name = Column(String)

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    content = Column(String)
    rating = Column(Integer)
    author_id = Column(Integer, ForeignKey('authors.id'))

    author = relationship('Author', back_populates='articles')

Author.articles = relationship('Article', order_by=Article.id, back_populates='author')

In [7]:
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///:memory:')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# Querying articles with their authors
articles_with_authors = session.query(Article).join(Author).all()
for article in articles_with_authors:
    print(f'Title: {article.title}, Author: {article.author.name}')

In [51]:
from sqlmodel import Field, SQLModel, select
import inspect
from sqlalchemy.dialects.postgresql import UUID as PG_UUID
from sqlalchemy.dialects.postgresql import JSONB
from enum import Enum
from typing import TypeVar
from pydantic import BaseModel, ConfigDict, create_model
from uuid import UUID, uuid4
from sqlalchemy.sql.expression import func
from asyncstdlib.functools import CachedProperty

In [53]:
from sqlalchemy import Integer, String, MetaData
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import mapped_column


class Base(DeclarativeBase):
    pass


class User(Base):
    __tablename__ = "user"

    id = mapped_column(Integer, primary_key=True)
    name = mapped_column(String(50), nullable=False)
    fullname = mapped_column(String)
    nickname = mapped_column(String(30))

In [55]:
# equivalent Table object produced
user_table = Table(
    "user",
    Base.metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String(50)),
    Column("fullname", String()),
    Column("nickname", String(30)),
    extend_existing=True
)

In [42]:
def SimpleForeignKey(  # noqa: N802
    column: str,
    ondelete:str  = "CASCADE",
    column_type: None = None,
    *args,
    **kwargs,
) -> Field:
    """
    Shortcut for the foreign key that easily allow to specify `ondelete` behaviors.

    HELPER - ondelete arg :
    ondelete is used in ForeignKey class from SQL Alchemy:
    - CASCADE: When the referenced row is deleted, all rows containing this foreign key will also be deleted.
    - SET NULL: When the referenced row is deleted, the value of the foreign key in the dependent rows will be set to NULL.
    - RESTRICT: Prevents the deletion of the referenced row if dependent rows exist.
    - NO ACTION: No specific action is taken. It depends on the default behavior of the database.
    - SET DEFAULT: When the referenced row is deleted, the value of the foreign key in the dependent rows will be set to a default value.
    """
    # Set nullable & default values 
    params = (
        {"nullable": False}
    )
    class_name = inspect.stack()[1][3] # Retrieve Class Name (e.g. if called by Class Collection, retrieve Collection)
    field_name = inspect.stack()[1][4][0].split(":")[0].strip()
    # 63 is the upper limit for constraint names
    fk_name = f"fk_{class_name.lower()}_{field_name}_to_{column}"[:63]
    print(fk_name, class_name, field_name)
    return Field(
        *args,
        sa_column=Column(
            column_type or PG_UUID(),
            ForeignKey(column, ondelete=ondelete, name=fk_name),
            **params,
        ),
        **kwargs,
    )

In [43]:
class EmbedModel(str, Enum):
    TEXT_EMBEDDING_ADA_002 = "text-embedding-ada-002"
    TEXT_EMBEDDING_3_LARGE = "text-embedding-3-large"
    TITAN_EMBEDDING_V2_0 = "amazon.titan-embed-text-v2:0"
    TITAN_EMBEDDING_V1 = "amazon.titan-embed-text-v1"

In [44]:
T = TypeVar("T", bound="BaseSQLModel")


def temporary_alias(string: str) -> str:
    if string == "id":
        return "uuid"
    return string.removesuffix("_id")


class BaseSQLModel(SQLModel):
    # Attribut un alias de id à uuid
    model_config = ConfigDict(
        extra="forbid", alias_generator=temporary_alias, ignored_types=(CachedProperty,)
    )

    id: UUID = Field(
        primary_key=True,
        default_factory=uuid4,
        unique=True,
        sa_column_kwargs={"server_default": func.gen_random_uuid()},
    )

In [45]:
# Gère les requêtes à la BD Postgre SQL + MAJ des variables API
class Collection(BaseSQLModel, table=True):
    # 4 != EmbedModel available in src.collections.enumps.py
    embed_model: EmbedModel = EmbedModel.TEXT_EMBEDDING_ADA_002
    name: str = "Coll 1"
    organization_id: UUID = SimpleForeignKey("organization.id")
    allowed_tags: dict = Field(default_factory=dict, sa_type=JSONB)
    auto_tag: bool = Field(default=False)


fk_collection_organization_id_to_organization.id Collection organization_id


InvalidRequestError: Table 'collection' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [46]:
Collection(name="test")

Collection(name='test', id=UUID('f040a87a-0f5a-4093-be70-35c72901f20e'), embed_model=<EmbedModel.TEXT_EMBEDDING_ADA_002: 'text-embedding-ada-002'>, organization_id=(FieldInfo(annotation=NoneType, required=True), 'Collection', 'organization_id'), allowed_tags={}, auto_tag=False)

In [15]:
field, class_name, field_name = SimpleForeignKey('organization.id')

In [16]:
print(field)

annotation=NoneType required=True


In [17]:
class_name

'<module>'